# `app.py`

While `main_window.py`'s responsibility is to **define the graphics user interface**, `app.py`'s responsibility is to **define the functionalities of the GUI**. This is achieved by doing 2 things:

1. Defining `functions` to accomplish certain actions
2. Connecting `Widget` actions to these `functions`

For example, if we want the `Update Window Button` to plot the stock prices in the GUI's canvas. We will have to create a `function` that plots the graph into the canvas and then connect the `Update Window Button` to this `function`.

However, before doing so, `app.py` must first know the `Widget names` defined in `main_window.py`. 

For example, the `Update Window Button` is actually named: `updateWindowButton`. This name is defined on the previous section, when `main_window.ui` was designed using `Qt Designer` and the `objectName` is specified inside the Property Editor!

This is why, on the previous step, it is **recommended** to name the `Widgets` **accordingly!**

This section of the report will go through the 3 steps of developing `app.py` + 1 optional step to compile `app.exe`, as summarized in the graphics below.

![app.py development process](asset/img/app-process.png)

## Inheriting `Widgets` from `main_window.py` 

The goal of this section is to ensure that `app.py` is **runnable without any error** and shows the **exact same** GUI as if previewing `main_window.ui`.

![app.py running correctly](asset/img/gui.png)

This result shows that `app.py` has successfully inherited all the properties of `main_window.py`, which includes all the `Widgets` defined when `main_window.ui` was created! These `Widgets` include `updateWindowButton`, `SMA1Checkbox`, `filePathEdit`, etc...

To achieve this, simply start from the generic starter code for all `PyQt5` application and then add the following:

1. Import `matplotlib`, `PyQt5` and the GUI's Widget class called `UI_Form` from `main_window`
2. Pass `QWidget` and `UI_Form` as argument to `Main` class to specify inheritance from `QWidget` and `UI_Form` class
3. Call the superclass' (`UI_Form`) initializing function and setup function
4. Finally, after the inherited GUI has been initialized, it is still possible to add other `Widgets` programmatically as well

This is exactly shown in the code below, running them should result in the image shown above:

In [ ]:
import sys
from pathlib import Path
from datetime import datetime

# Step 1
# standard matplotlib import statements
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# import matplotlib backend for Qt5
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

# standard PyQt5 import statements
from PyQt5 import QtCore as qtc
from PyQt5 import QtWidgets as qtw

# importing the class to be inherited from
from main_window import Ui_Form

# importing StockData processing module
from stock_data import StockData

class Main(qtw.QWidget, Ui_Form): # Step 2
    def __init__(self):
        # Step 3
        # calling Ui_Form's initializing and setup function
        super().__init__()
        self.setupUi(self)
        self.setWindowTitle("Stock Chart & Moving Average Application")

        # Step 4
        # sets up figure to plot on, instantiates canvas and toolbar
        self.figure, self.ax = plt.subplots()
        self.canvas = FigureCanvas(self.figure)
        self.toolbar = NavigationToolbar(self.canvas, self)

        # attaches the toolbar and canvas to the canvas layout
        self.canvasLayout.addWidget(self.toolbar)
        self.canvasLayout.addWidget(self.canvas)
        
        # sets up a scroll area to display GUI statuses
        self.scrollWidget = qtw.QWidget()
        self.scrollLayout = qtw.QVBoxLayout()
        self.scrollWidget.setLayout(self.scrollLayout)
        self.scrollArea.setWidget(self.scrollWidget)

if __name__ == "__main__":
    app = qtw.QApplication([])
    main = Main()
    main.show()
    sys.exit(app.exec_())

> *Learning Point: Inheriting `Widgets` from `main_window.py`*

> *When `main_window.ui` is converted into `main_window.py` using `pyuic5`, the `Widget` class called `Ui_Form` is created.*
> *This `Ui_Form` class has access to all the `Widgets` previously defined inside `main_window.ui` using `Qt Designer`!*
> *They're accessible to `Ui_Form` as regular python `Attributes`. e.g: `self.updateWindowButton`, etc...*
> *Thus, by inheriting from `Ui_Form`, `app.py`'s `Main` class can also access these `Widgets` through its `Attributes`.*
> *LIkewise, `functions` defined in `Ui_Form` are also inherited and accessible to `Main`*.

> *Learning Point: Defining & Adding `Widgets` programmatically*

> *Sometimes, it is more convenient to define `Widgets` programmatically then through `Qt Designer`.*
> *As shown from the code snippet above, this is also possible and uses the **exact same core principles** as in `main_window.py`*
> 1. *Defining each `Widget` objects' and their names within the GUI.*
> *Exemplified with lines such as:*
> *`self.canvas = FigureCanvas(self.figure)` or similar instantiation line: `button = QPushButton('Button Name', self)`*
> 2. *Defining the location, size and other physical attributes of each `Widgets`.*
> *Exemplified with lines such as:*
> *`self.canvasLayout.addWidget(self.canvas)`*

Now that `app.py` is able to access the `Widgets` defined in `main_window.py` by means of Python inheritance. It is now possible to implement `app.py`'s main responsibility:

> 1. Defining `functions` to accomplish certain actions
> 2. Connecting `Widget` actions to these `functions`

## Defining `functions` in `app.py`

Before defining the `functions` in `app.py`, it is important to first be aware of the scope of each `functions` needed to execute the app's entire process. By referring to the User Manual's 5-step guide, it is possible to breakdown the entire app's functionalities into 3 major functions + 2 minor functions:

1. `load_data(self)` : invoked when `Load CSV File Button` is pressed

> loads stock data .csv from inputted filepath string on the GUI as StockData object,
> also autocompletes all inputs using information provided by the csv.
> (Handles the actions from Step 1-2 of User Manual).

2. `update_canvas(self)` : invoked when `Load Update Window Button` is pressed

> creates a datetime object from the inputted date string of format YYYY-MM-DD.
> uses it to slice a copy of loaded stock_data to be used to update graphics.
> checks checkboxes first to see if SMA1, SMA2, Buy and Sell plots need to be drawn.
> finally, updates graphic accordingly.
> (Handles the actions from Step 3-5 of User Manual).

3. `plot_graph(self, column_headers, formats)` : invoked when `update_canvas function` is called

> plots graphs specified under columnd_headers using the formats specified
> (Helps to handle the action from Step 5 of User Manual).

4. `report(self, string)` : invoked when any of the 3 major functions are called

> given a report (string), update the scroll area with this report

5. `center(self)` : invoked `__init__(self)` is called (i.e. during the startup of app)

> centers the fixed main window size according to user screen size

The following part of the report will attempt to explain each of these 5 functions in detail. However, due to space limitation and the need for conciseness, only **parts of the code with its line number will be referenced!** We highly recommend that readers refer to the **full code in the Appendix or the python file itself should it become necessary**.

### `load_data(self)`

First, this function attempts to parse the `text` specified by user in the `Line Edit Widget` called `filePathEdit `for a `filepath` .

```
102 filepath = Path(self.filePathEdit.text())
```

> *Learning Point: Getting `Line Edit Widget` Value*

> *To extract the `string` value from `Line Edit Widget`, use: `.text()` method*

 The parsing of this `filepath` is outsourced to Python's `pathlib` library.

> *Learning Point: Using `Path` from `pathlib` to parse `filepath`*

> *To parse the `filepath` from `string`, simply use the standard python `pathlib`.*
> *Instantiate a `Path` object by passing the `string` as follows: `Path(string)`.*
> *This guarantees that the resultant `filepath` follows the proper format that the computer OS uses.*

Next, it will attempt to instantiate a `StockData` data object using this `filepath`. However, to prevent crashes due to invalid `filepath` or `.csv` file, it is important to wrap the previous instantiation line with a `try... except...`.

```
104 try:
105 	self.stock_data = StockData(filepath)
...
121 except IOError as e:
122 	self.report(f"Filepath provided is invalid or fail to open .csv file. {e}")
123 
124 except TypeError as e:
125 	self.report(f"The return tuple is probably (nan, nan) because .csv is empty")
```
Each of this `except` corresponds to the the errors mentioned in the function's docstring line 96 to 100.

> *Learning Point: Preventing Crashes with `try... except...`*

> *To prevent crashes, simply encapsulate the line inside a `try... except...`.*
> *Each type of error can then be handled individually.*

Once `StockData` has been initialized, the function attempts to get the `start_date` and `end_date` of the `stock_data` by `StockData`'s method called `get_period()`.

```
106 	start_date, end_date = self.stock_data.get_period()
107 	period = f"{start_date} to {end_date}"
```

Finally, the function will attempt to 'auto-complete' the various `Widgets` using information such as the `start_date` and `end_date`.

```
109 	# auto-complete feauture
110 	self.startDateEdit.setText(start_date)
111 	self.endDateEdit.setText(end_date)
112 	self.periodEdit.setText(period)
113 	self.SMA1Edit.setText("15")
114 	self.SMA2Edit.setText("50")
115 	self.SMA1Checkbox.setChecked(False)
116 	self.SMA2Checkbox.setChecked(False)
```

> *Learning Point: Setting `Widget` Values Programmatically.*

> *To set values to `Widgets` there are various methods specific to each type of `Widget`.*
> *`Line Edit Widget` uses `.setText(string)` whereas `Checkbox Widget` uses `.setChecked(bool)`.*

### `update_canvas(self)`

### `plot_graph(self, column_headers, formats)`

### `report(self, string)`

### `center(self)`

## Connecting `Widget` actions to `functions`

Fortunately, connecting `Widget` actions to `functions`are much simpler than defining the `functions`. These are all done inside the `__init__(self)` function. i.e. The app will attempt to connect these functions when it is first initialized/started by the user. 

The method used to connect `Widgets` to `functions` is: `Widget.connect(function)`

Simply add the following code to the the starter code given in section: "Inheriting `Widgets` from `main_window.py`" to complete `app.py`.

#### `__ init __(self)`

```
...
81 # button & checkbox connections
82 self.loadCSVButton.clicked.connect(self.load_data)
83 self.updateWindowButton.clicked.connect(self.update_canvas)
84 self.SMA1Checkbox.stateChanged.connect(self.update_canvas)
85 self.SMA2Checkbox.stateChanged.connect(self.update_canvas)
86 
87 # auto-complete feauture
88 self.filePathEdit.setText("../data/GOOG.csv")
```

> *Learning Point: Connecting `Widgets` to `Functions`*

> *To connect `Widgets` to `functions` use the following method: `Widget.connect(function)`.*
> *This ensures that when users interact with the `Widget` e.g. by pressing `Button`, checking `Checkbox`, etc..., it will trigger the appropriate functions*

## (Optional) Compiling `app.exe`

To compile `app.py` application into an executable, first install `pysinstaller` using `PIP` by running the following command:
```
pip install pyinstaller
```

Having installed `pyinstaller`, then use the following command from `root` folder:
```
pyinstaller .\src\app.py -F
```
The `app.exe` file can be found inside the `dist` folder. 

Note: the above command assumes that all source code (such as `app.py`, `stock_data_py` and `main_window.py`) are all found inside the `src` folder!

`app.exe` is a binary executable file for Windows (not Mac!). It allows users to simply double-click this file to start the application without requiring installation of any python modules at all.

> *Learning Point: Compiling Python Modules into an `.exe`*

> *PyInstaller is a standard package to bundle a Python application and all of its dependencies into a single executable.* 
> *The user can then run the packaged app without installing a Python interpreter or any modules.*
> *However, this is only possible for Windows!*

![The End](asset/img/the-end.png)